# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [46]:
import pandas as pd
import numpy as np
import folium


In [47]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: False


In [48]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [49]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [50]:
from sklearn.cluster import KMeans

In [51]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [52]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [53]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [54]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [55]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [56]:
# first, use the model to predict the cluster assignment of all data points
predictions = myKMeans.predict(coordinates.to_numpy())

In [57]:
#result gives a cluster number per coordiante
# look at the first 10 entries
predictions[:10]

array([49, 85, 63, 46, 64, 54, 95, 71, 79, 46], dtype=int32)

In [58]:

def show_cluster(cluster_number, predictions, centers):
    #get coordinates of the target cluster
    #coordinates were all coordinate collumns from the train data
    cluster_coord = coordinates.to_numpy()[predictions==cluster_number]
    
    #get number of entries:
    entries = np.shape(cluster_coord)[0]
    print("number of cluster enries:" , entries)
    
    #create map
    cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
    
    #plot centers
    folium.PolyLine([ [centers[cluster_number,0],centers[cluster_number,1]] , [centers[cluster_number,2],centers[cluster_number,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)
    
    for i in range(entries):
        folium.CircleMarker([cluster_coord[i,0], cluster_coord[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
        folium.CircleMarker([cluster_coord[i,2], cluster_coord[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    return cluster_map

In [65]:
show_cluster(57,predictions,centers)

number of cluster enries: 101


## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.

In [62]:
def show_var(predictions, k):
    #get coordinates of the target cluster
    #coordinates were all coordinate collumns from the train data
    np_coord = coordinates.to_numpy()
    
    #compute min and max coordiantes for normalization
    min_cor = np.min(np_coord, axis=0)
    max_cor = np.max(np_coord, axis=0)
    dist = max_cor - min_cor
    print ("max distances:", dist)
    for i in range(k):
        cluster_coord = np_coord[predictions==i]
    
        #compute variances
        var_all = np.var(cluster_coord, axis=0)
        print(i, var_all)

In [63]:
show_var(predictions,100)

max distances: [1.073698 1.177225 0.970017 1.169471]
0 [2.80520571e-05 3.72260004e-05 3.81155485e-05 3.70986779e-05]
1 [4.61910249e-05 2.84574540e-05 3.64858055e-05 2.79837442e-05]
2 [0.00013355 0.00016322 0.00033823 0.00059742]
3 [4.48357205e-05 1.90205968e-04 4.16875019e-04 3.47979992e-04]
4 [9.68515389e-05 7.90329567e-04 1.18990733e-06 4.81022113e-04]
5 [2.38422563e-04 1.65354369e-04 5.25604957e-05 6.95088829e-05]
6 [3.11200969e-05 1.50744675e-05 3.15886704e-05 4.15681676e-05]
7 [3.21152363e-05 4.67202045e-05 3.53651928e-05 2.82466152e-05]
8 [2.35405424e-04 1.87787254e-04 5.71303683e-05 9.55818030e-05]
9 [0.00024095 0.00011275 0.00029078 0.00024506]
10 [8.80168182e-05 1.06245611e-04 3.40347275e-04 8.63288486e-05]
11 [0.00293911 0.00454012 0.00098063 0.00242336]
12 [5.24517289e-05 5.03808665e-05 4.78745091e-05 3.12224919e-05]
13 [0.00020215 0.00017868 0.00020675 0.00021285]
14 [1.12521395e-04 7.81177575e-05 1.20527321e-04 1.04166642e-04]
15 [0.00417462 0.01161294 0.00444721 0.0100846